In [4]:
# -*- coding: utf-8 -*-
"""
Created on Tue Apr 20 21:05:52 2021

@author: ELM21
"""

import csv
import numpy as np
import sklearn
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import operator
from sklearn.preprocessing import StandardScaler


test = pd.read_csv('test_features.csv')
features = pd.read_csv('train_features.csv') #all features from csv
labels = pd.read_csv('train_labels.csv') # labels for all subtasks
task1labels = list(labels)[1:11] #labels for subtasks 1,2,3
task2labels = ['LABEL_Sepsis']
task3labels = list(labels)[12:]
allnames = list(features)
names = list(features)[3:]
scaler = StandardScaler()


#"crunch dataset" takes dataset and get, min for each patient each entry, variance and mean
def transformdata(data):

	A = data.groupby(np.arange(len(data))//12).min()
	B = data.groupby(np.arange(len(data))//12).max()
	C = data.groupby(np.arange(len(data))//12).var()
	D = data.groupby(np.arange(len(data))//12).mean()
	F = data.groupby(np.arange(len(data))//12).last()

	anames = ['min of ' + s for s in names]
	bnames = ['max of ' + s for s in names]
	cnames = ['var of ' + s for s in names]
	dnames = ['pid','Time','Age']+['mean of ' + s for s in names]
	fnames = ['last measurement of ' +s for s in names]


	A = A.drop(['pid','Time','Age'],axis =1)
	B = B.drop(['pid','Time','Age'],axis =1)
	C = C.drop(['pid','Time','Age'],axis =1)
	F = F.drop(['pid','Time','Age'],axis =1)


	A.columns=anames
	B.columns=bnames
	C.columns=cnames
	D.columns = dnames
	F.columns = fnames

	E = pd.concat([D,F,A,B,C],axis=1)
	E = E.drop(['pid','Time'],axis=1)

	#E.interpolate(method = "akima", inplace= False)
	imputer = SimpleImputer(missing_values=np.nan,strategy = 'mean')

	imputer.fit(E)
	E = imputer.transform(E)

	print("imputation finished")
    
	return E


def predict1(fulldataset,fulltestset):
           
   
    Y = labels[task1labels]
    
    
    #fulldataset = scaler.fit_transform(fulldataset)
    #fulltestset = scaler.transform(fulltestset)
    
    rfc = RandomForestClassifier(n_estimators=1000,bootstrap=True,max_features = 'sqrt',oob_score=True,n_jobs=-1,random_state=1, class_weight = 'balanced_subsample')
    rfc.fit(fulldataset,Y)
    
    print(rfc.oob_score_)
    print('score at 1')
    

    y1=rfc.predict_proba(fulltestset)
    return y1
        
def predict2(fulldataset,fulltestset):
           
   
      
    Y = labels[task2labels]
    #fulldataset = scaler.fit_transform(fulldataset)
    #fulltestset = scaler.transform(fulltestset)


    rfc = RandomForestClassifier(n_estimators=1000,bootstrap=True,max_features = 'sqrt',oob_score=True,n_jobs=-1,random_state=1, class_weight="balanced_subsample")
    rfc.fit(fulldataset,np.ravel(Y))
    
    print(rfc.oob_score_)
    print('score at 2')
    
    y2=rfc.predict_proba(fulltestset)
    return y2

def predict3(fulldataset,fulltestset):
    
    Y=labels[task3labels]
    
    fulldataset = scaler.fit_transform(fulldataset)
    fulltestset = scaler.transform(fulltestset)

    lasso_params = {'alpha' : np.linspace(0.01,0.03, 4)}
    ridge_params = {'alpha': np.linspace(200, 500, 4)}

    models = {'OLS': linear_model.LinearRegression(),
    		'Lasso': GridSearchCV(linear_model.Lasso(), param_grid = lasso_params).fit(fulldataset, Y).best_estimator_,
    		'Ridge': GridSearchCV(linear_model.Ridge(), param_grid = ridge_params).fit(fulldataset, Y).best_estimator_}
    
    scores = {}
    for model in models:
      print("PASS")
      score = np.mean(cross_val_score(models[model], fulldataset, Y, scoring='r2', cv = 3))
      scores[model] = score
    final_model = max(scores.items(), key = operator.itemgetter(1))[0]

    md = models[final_model]

    y3 = md.predict(fulltestset)

    return y3

def getids(dataset):
    pid = dataset['pid']
    pid = pid.groupby(np.arange(len(dataset))//12).mean()

    return pid

def getpredictions(dataset,testset):
    

    X = transformdata(dataset)
    y = transformdata(testset)

    y1 = predict1(X,y)
    y2 = predict2(X,y)
    y3 = predict3(X,y)

    y1 = np.array(y1)
    y1 = y1[:,:,1]
    y1 = np.transpose(y1)
    y2 = y2[:,1]

    print(np.shape(np.array(y2)))
    print(np.shape(np.array(y1)))
    
    yall = np.column_stack((y1,y2,y3))
    ids = getids(testset)
    yall = np.column_stack((ids,yall))
    yall =pd.DataFrame(yall, columns = list(labels))

    yall.to_csv('predictions.csv',index=False,float_format = '%.3f')
    
getpredictions(features,test)


imputation finished
imputation finished
0.8518662806001579
score at 1
0.9427217688865491
score at 2
PASS
PASS
PASS
(12664,)
(12664, 10)
